# Sentiment Analysis

In [3]:
# Requirements
# Install PyTorch use the line below in command prompt

# pip3 install torch torchvision torchaudio

# Ensure you have the most up to date versio of pip by running
# py -m pip install --upgrade pip

# Install virtualenv to prevent global conflicts between package versions. Install by running:
# py -m pip install --user virtualenv

> Import Packages

In [1]:
# Data import and manipulation
import pandas as pd

# Using hugging face package 'transformers' load function 'pipeline'
from transformers import pipeline

ModuleNotFoundError: No module named 'transformers'